In [2]:
import tkinter as tk
from tkinter import messagebox, filedialog
import boto3

# Initialize Boto3 client for S3
s3_client = boto3.client('s3')

# Function to list objects in a selected S3 bucket
def list_objects_in_bucket(bucket_name, objects_listbox):
    try:
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        if 'Contents' in response:
            objects = [obj['Key'] for obj in response['Contents']]
            for obj in objects:
                objects_listbox.insert(tk.END, obj)
    except Exception as e:
        messagebox.showerror("Error", f"Failed to list objects in bucket: {str(e)}")

# Function to upload file to the selected S3 bucket
def upload_to_s3_bucket(bucket_name, objects_listbox):
    def upload():
        file_path = filedialog.askopenfilename()
        if file_path:
            key = file_path.split('/')[-1]  # Using filename as key
            try:
                s3_client.upload_file(file_path, bucket_name, key)
                messagebox.showinfo("Success", "File uploaded successfully!")
                objects_listbox.insert(tk.END, key)  # Update objects listbox
            except Exception as e:
                messagebox.showerror("Error", f"Failed to upload file: {str(e)}")
    return upload

# Function to delete object from the selected S3 bucket
def delete_from_s3_bucket(bucket_name, objects_listbox):
    def delete():
        selected_object_index = objects_listbox.curselection()

        if not selected_object_index:
            messagebox.showerror("Error", "Please select an object to delete.")
            return

        selected_object_index = selected_object_index[0]
        selected_object_key = objects_listbox.get(selected_object_index)

        try:
            s3_client.delete_object(Bucket=bucket_name, Key=selected_object_key)
            messagebox.showinfo("Success", "Object deleted successfully!")
            objects_listbox.delete(selected_object_index)  # Update objects listbox
        except Exception as e:
            messagebox.showerror("Error", f"Failed to delete object: {str(e)}")
    return delete

# Function to handle bucket selection event
def on_bucket_select(bucket_name):
    # Open a new window for bucket operations
    bucket_window = tk.Toplevel(root)
    bucket_window.title(f"{bucket_name} Bucket Operations")

    objects_label = tk.Label(bucket_window, text=f"Objects in {bucket_name}")
    objects_label.pack()

    objects_listbox = tk.Listbox(bucket_window, selectmode=tk.SINGLE)
    objects_listbox.pack(fill=tk.BOTH, expand=True)

    # Populate objects listbox
    list_objects_in_bucket(bucket_name, objects_listbox)

    # Upload button
    upload_button = tk.Button(bucket_window, text="Upload", command=upload_to_s3_bucket(bucket_name, objects_listbox))
    upload_button.pack(pady=5)

    # Delete button
    delete_button = tk.Button(bucket_window, text="Delete", command=delete_from_s3_bucket(bucket_name, objects_listbox))
    delete_button.pack(pady=5)

# Main window
root = tk.Tk()
root.title("AWS S3 Manager")

buckets_label = tk.Label(root, text="S3 Buckets")
buckets_label.pack()

buckets_listbox = tk.Listbox(root, selectmode=tk.SINGLE)
buckets_listbox.pack(fill=tk.BOTH, expand=True)

# Populate buckets list
try:
    response = s3_client.list_buckets()
    buckets = [bucket['Name'] for bucket in response['Buckets']]
    for bucket in buckets:
        buckets_listbox.insert(tk.END, bucket)
except Exception as e:
    messagebox.showerror("Error", f"Failed to list S3 buckets: {str(e)}")

# Bind bucket selection event
def on_bucket_selection(event):
    selected_bucket_index = buckets_listbox.curselection()
    if selected_bucket_index:
        selected_bucket_index = selected_bucket_index[0]
        selected_bucket_name = buckets_listbox.get(selected_bucket_index)
        on_bucket_select(selected_bucket_name)

buckets_listbox.bind("<<ListboxSelect>>", on_bucket_selection)

root.mainloop()